In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [2]:
# 显示cell运行时长
%load_ext klab-autotime

In [1]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

In [2]:
!pip install matplotlib==2.2.2 

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
# df_train = pd.read_csv('/home/mw/input/tianchi6504/model_sample.csv')
df_train = pd.read_csv('/home/mw/work/features.csv',index_col =0)
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_184/x_179,x_185/x_180,x_189/x_188,x_191/x_190,x_193/x_192,x_195/x_194,x_197/x_196,x_199/x_198,x_196/x_188,x_192/x_188
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,0.25000,1000.000000,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,0.62069,1606.939091,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,0.00000,1570.506667,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [4]:
df_train.y.value_counts()

0    8873
1    2144
Name: y, dtype: int64

In [5]:
# -*- coding: utf-8 -*-
import numpy as np  
import matplotlib.mlab as mlab  
import matplotlib.pyplot as plt  
labels=['未逾期','逾期']

 
fig = plt.figure()
plt.pie(df_train.y.value_counts(),labels=labels,autopct='%1.2f%%') #画饼图（数据，数据对应的标签，百分数保留两位小数点）
plt.title("Pie chart")
  
 
plt.show()  
plt.savefig("逾期分布")

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 0 Axes>

In [6]:
no_features = ['y','user_id']
features = [col for col in df_train.columns if col not in no_features]

In [7]:
# train_X,test_X, train_y, test_y = train_test_split(model_sample,label,test_size=0.2,random_state=2021) 
    
seed =2021

In [8]:
# lgb_train = lgb.Dataset(train_X, train_y)
# lgb_eval = lgb.Dataset(test_X, test_y, reference=lgb_train)

In [9]:
# params = {
#             'task':'train',
#             'boosting_type':'gbdt',
#             'num_leaves': 31,
#             'objective': 'binary', 
#             'learning_rate': 0.01, 
#             'bagging_freq': 2, 
#             'max_bin':256,
#             'num_threads': 4,
#              'metric' : ['auc', 'binary_logloss']
           

#         } 

        # train


In [10]:
# gbm = lgb.train(params,
#                     lgb_train,
#                     verbose_eval= 100,
#                     num_boost_round=10000,
                   
#                     valid_sets=lgb_eval,
#                     early_stopping_rounds=100)

In [39]:
# 五折交叉验证
ycol = 'y'
feature_names = features
# 贝叶斯调参数最优
#  {'bagging_fraction': 0.8256838817068106,
#   'bagging_freq': 12.897449911782854,
#   'feature_fraction': 0.6799417915036878,
#   'max_depth': 19.607609424498186,
#   'min_child_weight': 7.989484773056548,
#   'min_data_in_leaf': 92.87862688573793,
#   'min_split_gain': 0.5653189848881633,
#   'num_leaves': 178.8305250513094,
#   'reg_alpha': 9.992751318598641,
#   'reg_lambda': 1.2535560707642368}
model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                         bagging_fraction = 0.82,
                        bagging_freq =  12,
                      feature_fraction = 0.67,
                        max_depth =  19,
                min_child_weight= 7.989484773056548,
                min_data_in_leaf= 92,
                min_split_gain= 0.56,
                num_leaves= 178,
                reg_alpha= 9,
                reg_lambda= 1,
                           learning_rate=0.01,
                        
                           random_state=seed,
                           is_unbalance=True,
                           metric=['auc','binary_logloss','f1'])


df_oof = []
df_importance_list = []
lgb_models = []
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names],df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))
    # print(df_oof.loc[val_idx]['prob'])
    
    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          early_stopping_rounds=50)
    lgb_models.append(lgb_model)
    

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
   
    df_oof = df_train.iloc[val_idx][[
        'user_id', ycol]].copy()
    df_oof['pred'] = pred_val
    df_oof.append(df_oof)
    

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model,  X_train, Y_train, X_val, Y_val
    gc.collect()
lgb.create_tree_digraph(lgb_models[-1], tree_index=1)


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[47]	train's binary_logloss: 0.448864	train's auc: 0.819389	valid's binary_logloss: 0.456551	valid's auc: 0.781887

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[46]	train's binary_logloss: 0.448576	train's auc: 0.816522	valid's binary_logloss: 0.457246	valid's auc: 0.779998

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[49]	train's binary_logloss: 0.450263	train's auc: 0.814213	valid's binary_logloss: 0.45694	valid's auc: 0.795398

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[49]	train's binary_logloss: 0.447859	train's auc: 0.819344	vali

In [40]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()


In [41]:
df_importance.head(10)

,column,importance
0,x_196/x_188,80.4
1,x_035/all_cards,50.4
2,x_125/x_121,49.8
3,x_034/all_cards,47.6
4,x_020,44.8
5,x_143/x_144,44.2
6,x_146,36.4
7,x_002,33.2
8,x_127,33.2
9,x_045/x_41,26.4


In [42]:
df_importance[df_importance.importance ==0].shape 

(116, 2)

In [43]:
df_importance[df_importance.importance ==0]

,column,importance
185,x_178/x_177,0.0
186,x_008,0.0
187,x_005,0.0
188,x_177,0.0
189,x_007,0.0
190,x_178,0.0
191,x_006,0.0
192,x_012,0.0
193,x_039,0.0
194,x_009,0.0


In [44]:
df_oof.to_csv('lgb_oof_feature_engineering.csv',index = False)

In [45]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
Threshold = 0.36

In [46]:
accuracy_score(df_oof['y'], df_oof['pred'] >Threshold)

0.7643051771117166

In [47]:
recall_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')


0.6853090855451012

In [48]:
f1_score(df_oof['y'], df_oof['pred'] >Threshold)


0.47839195979899496

In [49]:
precision_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')

0.6517725676747084

In [50]:
def get_roc_auc(y_test,y_score):
    # y_score =clf.predict_proba(test_vecs)[:,1]
    auc_s = roc_auc_score(y_test,y_score)   
    # print(y_predict)
    # print(y_score)
#     print(cm)
    print(auc_s)
    # Compute ROC curve and ROC area for each class
    fpr,tpr,threshold = roc_curve(y_test, y_score) ###计算真正率和假正率
    roc_auc = auc(fpr,tpr) ###计算auc的值
    
    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [51]:
get_roc_auc(df_oof['y'], df_oof['pred'])

0.7734988251904455


<Figure size 432x288 with 0 Axes>

<Figure size 720x720 with 1 Axes>

In [52]:
#coding:utf-8
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
# from sklearn.utils.fixes import signature
plt.figure("P-R Curve")
plt.title('Precision/Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
precision, recall, thresholds = precision_recall_curve(df_oof['y'], df_oof['pred'])
#print(precision)
#print(recall)
#print(thresholds)
plt.plot(recall,precision)
plt.show()

<Figure size 432x288 with 1 Axes>

In [53]:

fpr,tpr,thresholds= roc_curve(df_oof['y'], df_oof['pred'])
print ('KS:',max(tpr-fpr))

KS: 0.42757272756008385


In [54]:
def get_confusion_matrix(test_y,pred):
    f,ax=plt.subplots()

    C2= confusion_matrix(test_y, pred, labels=[0, 1])
    print(C2) #打印出来看看
    sns.set(font_scale=1.5)
    sns.heatmap(C2,annot=True,ax=ax, fmt='.20g',cmap="YlGnBu") #画热力图

    ax.set_title('confusion matrix') #标题
    ax.set_xlabel('predict') #x轴
    ax.set_ylabel('true') #y轴

In [55]:
get_confusion_matrix(df_oof['y'], df_oof['pred'] > 0.364)

[[1466  308]
 [ 204  224]]


<Figure size 432x288 with 2 Axes>

In [56]:
# (df_oof['pred'] >=0.36).astype('int64')

In [57]:
max_f1 = 0
Threshold = 0
for i in range(200,400):
    f1 = f1_score(df_oof['y'], df_oof['pred'] >i * 1e-3,average='macro')
    # f1 = f1_score(df_oof['y'].astype('int64'), (df_oof['pred'] >=i * 1e-3).astype('int64'),average='weighted')
    if max_f1 < f1:
        max_f1 = f1
        Threshold =  i

max_f1 ,Threshold * 1e-3

(0.666342572132079, 0.356)